#Install core libraries

In [ ]:
!pip install -q google-adk google-genai duckduckgo-search python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 39.5 MB/s eta 0:00:00


#Configure Gemini client and sanity check

In [ ]:
import os
from google import genai
from google.genai import types as genai_types

from google.adk.agents import LlmAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import FunctionTool

os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY
client = genai.Client(api_key=GEMINI_API_KEY)

# Quick sanity check
resp = client.models.generate_content(
    model="gemini-2.0-flash-001",
    contents="Say hello in one short sentence as a lead research assistant.",
)
print("Sanity check:", resp.text)


/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


Sanity check: Hello, I'm here to assist with the research.



#Quiet noisy warnings

In [9]:
import warnings

warnings.filterwarnings(
    "ignore",
    category=DeprecationWarning,
    module="jupyter_client.session"
)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

#Web search tool

In [10]:
!pip install -q ddgs

from ddgs import DDGS

def web_search(query: str, max_results: int = 5) -> list[dict]:
    results = []
    with DDGS() as ddgs:
        for r in ddgs.text(query, max_results=max_results):
            results.append(
                {
                    "title": r.get("title", ""),
                    "url": r.get("href", ""),
                    "snippet": r.get("body", ""),
                }
            )
    return results

print(web_search("B2B SaaS CRM", max_results=3))


[{'title': 'ChartMogul A smarter CRM for selling more SaaS | ChartMogul', 'url': 'https://chartmogul.com/crm/', 'snippet': 'For SaaS companies doing founder-led sales. $0 No credit card required · For sales, account management and success teams. $32.50 / seat Per month, billed annually · ChartMogul CRM is built specifically for B2B SaaS businesses.'}, {'title': 'Scalecrush The Top 10 CRM For SaaS In 2024 | ScaleCrush', 'url': 'https://www.scalecrush.io/blog/crm-saas', 'snippet': 'February 28, 2024 - Select a CRM that can scale alongside your business, offering flexible pricing plans, expanded feature sets, and increased capacity to accommodate growing user numbers and data demands. This will ensure that your CRM solution remains effective and efficient as your SaaS scales. Now let’s take a look at the best CRMs for B2B SaaS companies on the market today.'}, {'title': 'Kalungi 2025 guide to finding the right CRM for your B2B SaaS venture', 'url': 'https://www.kalungi.com/blog/marketing-

#Lead storage tool

In [11]:
import csv, os
from google.adk.tools import FunctionTool

LEADS_CSV = "leads.csv"

# Create file with header if it doesn't exist
if not os.path.exists(LEADS_CSV):
    with open(LEADS_CSV, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["company", "website", "segment", "why_good_lead"])

def save_leads(leads: list[dict]) -> str:
    """
    Save a list of leads to leads.csv.
    Each lead dict should have: company, website, segment, why_good_lead.
    """
    if not isinstance(leads, list):
        return "Expected a list of lead objects."

    with open(LEADS_CSV, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        for lead in leads:
            writer.writerow([
                lead.get("company", ""),
                lead.get("website", ""),
                lead.get("segment", ""),
                lead.get("why_good_lead", ""),
            ])
    return f"Saved {len(leads)} leads to {LEADS_CSV}."

save_leads_tool = FunctionTool(func=save_leads)


#Lead generation LlmAgent

In [12]:
test_leads = [
    {
        "company": "TestCo",
        "website": "https://example.com",
        "segment": "B2B SaaS CRM",
        "why_good_lead": "Matches ICP and has clear SaaS pricing.",
    }
]

print(save_leads(test_leads))


Saved 1 leads to leads.csv.


#Runner and session

In [13]:
from google.adk.agents import LlmAgent

deep_research_agent = LlmAgent(
    name="lead_generation_agent",
    model="gemini-2.0-flash-001",
    instruction=(
        "You are a deep research assistant for B2B lead generation.\n"
        "- Given a target segment, plan a research strategy.\n"
        "- Use the web_search tool to find companies and context.\n"
        "- Propose high-quality leads with: company, website, segment, why_good_lead.\n"
        "- When done, call save_leads(leads) ONCE with a list of lead objects.\n"
        "- Then summarize the final leads for the user."
    ),
    tools=[web_search_tool, save_leads_tool],
)


In [15]:
from google.adk.runners import InMemoryRunner
from google.genai import types as genai_types

APP_NAME = "lead_gen_app"
USER_ID = "demo-user"

runner = InMemoryRunner(agent=deep_research_agent, app_name=APP_NAME)

# 🔑 NOTE: use await here
session = await runner.session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
)

print("Session ID:", session.id)


Session ID: 3266a164-802b-4921-a845-2d6d98ce6043


#Run a lead-generation query

In [20]:
from google.genai.types import Content, Part

def run_lead_query(prompt: str):
    user_msg = Content(
        role="user",
        parts=[Part.from_text(text=prompt)],
    )

    print("=== USER PROMPT ===")
    print(prompt)
    print("\n=== AGENT RESPONSE ===")

    for event in runner.run(
        user_id=session.user_id,
        session_id=session.id,
        new_message=user_msg,
    ):
        # Only care about the final LLM response, not tool calls
        if hasattr(event, "is_final_response") and event.is_final_response():
            if event.content:
                for part in event.content.parts:
                    text = getattr(part, "text", None)
                    if text:
                        print(text.strip())

# try it again
run_lead_query("Find 3 promising B2B SaaS CRM startups for mid-market companies.")

/usr/local/lib/python3.12/dist-packages/google/adk/runners.py:1166: DeprecationWarning: deprecated
  save_input_blobs_as_artifacts=run_config.save_input_blobs_as_artifacts,


=== USER PROMPT ===
Find 3 promising B2B SaaS CRM startups for mid-market companies.

=== AGENT RESPONSE ===


Okay, I have saved the leads. Now I will summarize the leads for you.

**Summary of Potential Leads:**

*   **Salesflare:** Easy to use and inexpensive CRM, suitable for small B2B businesses. Strong Google Workspace integration. Website: [https://salesflare.com/](https://salesflare.com/). Segment: B2B SaaS CRM for mid-market.
*   **Pipedrive:** Easy-to-use and easy-to-setup CRM software for startups and small businesses. Website: [https://www.pipedrive.com/](https://www.pipedrive.com/). Segment: B2B SaaS CRM for mid-market.
*   **ActiveCampaign:** CRM for B2B companies designed to help deliver memorable customer experiences. Website: [https://www.activecampaign.com/](https://www.activecampaign.com/). Segment: B2B SaaS CRM for mid-market.


sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7a0f805fbd20>


#Inspect saved leads

In [21]:
import pandas as pd, os

if os.path.exists("leads.csv"):
    display(pd.read_csv("leads.csv"))
else:
    print("leads.csv not found.")


,company,website,segment,why_good_lead
0,TestCo,https://example.com,B2B SaaS CRM,Matches ICP and has clear SaaS pricing.
1,ChartMogul,https://chartmogul.com/crm/,B2B SaaS CRM for mid-market,CRM built specifically for B2B SaaS businesses...
2,LeadCRM,https://www.leadcrm.io/blog/crm-for-saas-compa...,B2B SaaS CRM for mid-market,Good fit for SaaS companies using LinkedIn for...
3,Chili Piper,https://startupsavant.com/startups-to-watch/b2...,B2B SaaS CRM for mid-market,Revenue acceleration platform that integrates ...
4,Salesflare,https://salesflare.com/,B2B SaaS CRM for mid-market,"Easy to use and inexpensive CRM, suitable for ..."
5,Pipedrive,https://www.pipedrive.com/,B2B SaaS CRM for mid-market,Easy-to-use and easy-to-setup CRM software for...
6,ActiveCampaign,https://www.activecampaign.com/,B2B SaaS CRM for mid-market,CRM for B2B companies designed to help deliver...
